In [ ]:
pip install -U langchain-groq

In [ ]:
from typing import TypedDict, List, Union, Annotated, Sequence
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [ ]:
document_content=""

In [ ]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage],add_messages]

In [ ]:
@tool
def update(content: str) -> str:
  """Updates the document with the provided content"""
  global document_content
  document_content = content
  return f"Document has been updates successfully. Content:{document_content}"

@tool
def save(filename:str)->str:
  """Save the current document to a text file and finish the process
  Args:
    filename: Name of the text file
  """
  if not filename.endswith('.txt'):
    filename=f"{filename}.txt"
  try:
    with open(filename, 'w') as file:
      file.write(document_content)
    print(f"\n💾 Document has been saved to: {filename}")
    return f"Document has been saved successfully to '{filename}'."
  except Exception as e:
    return f"Error saving document: {str(e)}"


In [ ]:
tools=[update,save]
llm = ChatGroq(
    groq_api_key="",
    model_name="llama-3.1-8b-instant"
).bind_tools(tools)

In [ ]:
def our_agent(state:AgentState)->AgentState:
  system_prompt = SystemMessage(content=f"""
  You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.

  - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
  - If the user wants to save and finish, you need to use the 'save' tool.
  - Make sure to always show the current document state after modifications.

  The current document content is:{document_content}
  """)
  if not state["messages"]:
    user_input = "I'm ready to help you update a document. What would you like to create?"
    user_message = HumanMessage(content=user_input)

  else:
    user_input = input("\nWhat would you like to do with the document? ")
    print(f"\n👤 USER: {user_input}")
    user_message = HumanMessage(content=user_input)

  all_messages = [system_prompt] + list(state["messages"]) + [user_message]
  response = llm.invoke(all_messages)

  print(f"\n🤖 AI: {response.content}")
  if hasattr(response, "tool_calls") and response.tool_calls:
    print(f"🔧 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

  return {"messages": list(state["messages"]) + [user_message, response]}

def should_continue(state: AgentState) -> str:
  """Determine if we should continue or end the conversation."""

  messages = state["messages"]

  if not messages:
    return "continue"

  for message in reversed(messages):
    if (isinstance(message, ToolMessage) and
        "saved" in message.content.lower() and
        "document" in message.content.lower()):
      return "end"

  return "continue"

def print_messages(messages):
  """Function I made to print the messages in a more readable format"""
  if not messages:
    return

  for message in messages[-3:]:
    if isinstance(message, ToolMessage):
      print(f"\n🛠️ TOOL RESULT: {message.content}")

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("agent", our_agent)
graph.add_node("tools",ToolNode(tools))

graph.set_entry_point("agent")

graph.add_edge("agent","tools")

graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue": "agent",
        "end": END
    }
)
app = graph.compile()

In [ ]:
def run_document_agent():
  print("\n ===== DRAFTER =====")

  state = {"messages": []}

  for step in app.stream(state, stream_mode="values"):
    if "messages" in step:
      print_messages(step["messages"])

  print("\n ===== DRAFTER FINISHED =====")

if __name__ == "__main__":
  run_document_agent()


 ===== DRAFTER =====

🤖 AI: We can create a new document from scratch or start with a template. For now, let's start with a basic template.

Current Document State:

{
  "title": "",
  "content": ""
}

What would you like to do first?

What would you like to do with the document? write a mail to acrorma regarding warranty claim on my psu

👤 USER: write a mail to acrorma regarding warranty claim on my psu

🤖 AI: Here's a draft email:

Current Document State:

{
  "title": "Warranty Claim for PSU",
  "content": "Subject: Warranty Claim for PSU\r\n\r\nDear ACRORMA Support Team,\r\n\r\nI am writing to submit a warranty claim for my Power Supply Unit (PSU) that malfunctioned. The details of the product are as follows:\r\n\r\nProduct Name: PSU\r\n\r\nSerial Number: \r\n\r\nDate of Purchase: \r\n\r\nI have followed all the guidelines and precautions mentioned in the user manual, but the product has stopped working after \r\n\r\nI would appreciate it if you could assist me in resolving this i